In [1]:
import jieba as jb
import nltk
import pandas as pd
from tqdm import tqdm
import ipyparallel as ipp
import jieba.posseg as pseg
#jb.set_dictionary('../data_build/cn_common/punct.txt')

In [2]:
jb.load_userdict('../data_build/cn_traditional_jieba/nr_nra_dict_CBDB.txt')
jb.load_userdict('../data_build/cn_traditional_jieba/ns_dict_CBDB.txt')
jb.load_userdict('../data_build/cn_traditional_jieba/no_noc_dict_CBDB.txt')
jb.load_userdict('../data_build/cn_traditional_jieba/td_tr_dict_CBDB.txt')
jb.load_userdict('../data_build/cn_traditional_jieba/nk_dict_CBDB.txt')
jb.load_userdict('../data_build/cn_traditional_jieba/nt_dict_CBDB.txt')
jb.load_userdict('../data_build/cn_common/punct.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.439 seconds.
Prefix dict has been built succesfully.


In [30]:
def replace_punct(string):
    string=string.replace('，', 'wsep').replace('；', 'wsep').replace('、', 'wsep')
    string=string.replace('！', 'wend').replace('。', 'wend').replace('？', 'wend').replace('！', 'wend')
    string=string.replace('：', 'wm')
    return [s for s in string.split("wend") if s!='']

In [104]:
df_qsw_raw=pd.read_excel('../data_raw/quan_song_wen_muzhi.xlsx', 
                         sheet_name='墓誌銘墓表壙誌行狀神道碑塔銘墓碑', 
                         encoding='utf8'
                        )
df_qsw_raw_sample=df_qsw_raw.sample(20)[['subject','content']].dropna()

In [108]:
df_qsw_raw_sample

,subject,content
1626,陳裕,某先父行己謹且信，不泛交游，與公相厚善，情好既篤，遂結爲姻家。今公之孫暘，以書告某曰：‘暘祖...
4433,曹豳,前闕悉以命汝。公益感慨，言：‘四十年間大火者三，辛酉、辛卯之火皆有兵禍，今兹之火安知非兵釁所...
3734,韓應,〔蓋文〕宋故贈右承議郎韓公墓誌銘〔誌文〕宋故通直郎守太子中舍知平定軍樂平縣事兼兵馬都監贈右承...
123,王平,慶曆乙酉，上用太原王公爲侍御史，數進方幅，切劘庶政。又明年，以天禧故事參領諫職，臺中雜細，一...
3707,范鍇妻,治平四年六月丁未，衛尉寺丞致仕贈太常博士范府君諱鍇之夫人曰永壽縣太君郭氏，卒於玉泉坊之第，熙...
1775,戴述,道學不明，世儒蔽聰明于方册文辭之間，不知反身入德之要。仁義禮智根于心，而措于事業，致懵昧于理...
233,尹洙,師魯河南人，姓尹氏，諱洙。然天下之士識與不識皆稱之曰師魯，蓋其名重當世。而世之知師魯者，或推...
2384,胡泳,君諱泳，字季永，吉州廬陵人。曾祖愷；妣張氏，以百歲封孺人。祖載，贈朝議大夫；妣碩人陳氏、張氏...
3783,富紹榮,蓋銘：宋奉直大夫富公墓銘朝散大夫前權利州路轉運判官蘇覺撰朝請郎新權發遣秦鳳等路提點刑獄公事王...
1331,酈仲隱女、宋仲妻,夫人酈氏，亳州衛真人。夫人幼敏惠，父仲隱奇之，欲以歸士大夫，而地寒未能自致也。故西京作坊使、...


In [ ]:
{sub:{kin1:{name}}}

In [101]:
grammer=r'''
         KIN1:
             {<nk><n.+>}
         KIN2:
             {<.+>+<wm><.+>+}
         '''
cp=nltk.RegexpParser(grammer)
string_list=replace_punct(df_qsw_raw_sample.iloc[8])
for string in string_list:
    sent = pseg.cut(string, HMM=True)
    kinship_string=cp.parse([(s,t) for s,t in sent])
    print(kinship_string)
    kin_list=[str(tree) for tree in kinship_string if str(tree).startswith('(KIN')]
    if len(kin_list)!=0:
        print(kin_list)

(S 君/n 諱/vg 守誠/nra wsep/wsep 字/n 亶/zg 夫/ng)
(S 先/d 宛丘/ns 人/n)
(S
  自/p
  秦王/no
  存審/v
  wsep/wsep
  當/zg
  五代/t
  諸侯/nr
  僭/g
  竊/vg
  wsep/wsep
  從/zg
  李克用/nr
  屢建/l
  奇功/a
  wsep/wsep
  後/nr
  事/n
  莊宗/nra
  wsep/wsep
  以/p
  將相/no
  位高/v
  天下/s
  wsep/wsep
  寔/yg
  君/v
  之/u
  六代/t
  祖/nr
  也/d)
(S
  及/nra
  顯德/tr
  初/nra
  wsep/wsep
  秦王/no
  以/p
  女孫册/n
  后/f
  wsep/wsep
  而/c
  苻氏/nr
  始/d
  世家/n
  於/nr
  京師/ns)
(S
  君/n
  即/v
  故/n
  如京使/no
  昭/vg
  矩/ng
  之/u
  曾孫/nk
  wsep/wsep
  内殿/n
  崇班/no
  wsep/wsep
  閤門祗候/no
  承/nra
  訓之/nra
  孫/nk
  wsep/wsep
  東頭供奉官/no
  wsep/wsep
  累贈/v
  左千牛衛大將軍/no
  惟熙/nra
  之/u
  子/ng
  wsep/wsep
  而/c
  追封/v
  仙居/ns
  縣太君/no
  畢氏/nr
  wsep/wsep
  其/r
  母/b
  也/d)
(S
  君/n
  幼/ag
  孤/ng
  wsep/wsep
  事/n
  仙居/ns
  至孝/v
  wsep/wsep
  有/v
  稱/v
  於/nr
  族/ng
  黨/n)
(S
  方在/nr
  童/nr
  齠/g
  wsep/wsep
  笑言/n
  必時/d
  wsep/wsep
  人/n
  皆/d
  卓然/nra
  以/p
  遠/a
  大期/n
  之/u)
(S
  長/zg
  wsep/wsep
  娶/v
  宗室/no
  定州/ns
  觀察使/no
  從質/nra
  之/u
  女/n
  ws

In [100]:
string_list

['君諱守誠wsep字亶夫',
 '先宛丘人',
 '自秦王存審wsep當五代諸侯僭竊wsep從李克用屢建奇功wsep後事莊宗wsep以將相位高天下wsep寔君之六代祖也',
 '及顯德初wsep秦王以女孫册后wsep而苻氏始世家於京師',
 '君即故如京使昭矩之曾孫wsep内殿崇班wsep閤門祗候承訓之孫wsep東頭供奉官wsep累贈左千牛衛大將軍惟熙之子wsep而追封仙居縣太君畢氏wsep其母也',
 '君幼孤wsep事仙居至孝wsep有稱於族黨',
 '方在童齠wsep笑言必時wsep人皆卓然以遠大期之',
 '長wsep娶宗室定州觀察使從質之女wsep恩補右班殿直wsep始獲試吏',
 '而飭己祗事wsep乃以年勞叙遷左班殿直wsep歷侍禁wsep供奉官',
 '既陞通籍wsep由内殿崇班改承制wsep又遷供備庫副使',
 '初監洪州武寧縣酒税wsep秩滿wsep主兩浙漕計者以姑蘇會郡wsep榷沽之利wsep歲入越十餘萬緡wsep求才力可辦者wsep莫如君焉wsep因薦任之',
 '次勾當京東窑務wsep又西京同巡檢wsep陝府兵馬都監wsep勾當車營致遠務',
 '即以疾致其仕wsep歸休於祥符縣永昌坊之里第',
 '亦既閲歲wsep終于正寢wsep時乃崇寧三年十月十二日也wsep享年六十有四',
 '君在陝郊wsep以前政貪穢狼藉之餘wsep兵籍謬紊wsep力役之任wsep更休失次wsep群情爲之汹然',
 '會君之來wsep一皆條革其弊wsep而衆用以安',
 '蓋忠厚之資wsep不苟操踐wsep憲令之外wsep非所持循wsep故所至皆取能譽',
 '方今士大夫之立朝者往往知君之才而論薦之wsep不幸白首wsep迄無所遇wsep不克大有施設',
 '疾既殛矣wsep命家人輩語以後事wsep且及諸族之未葬者wsep當自致資力wsep以襄其事wsep由是得從藏掩之惠者甚衆',
 '噫!君將殆矣wsep猶思以德愛人者如此wsep則君生平志意之所及者wsep固可得而知也',
 '子一人世美wsep右侍禁',
 '女三人wm長適皇叔坊州防禦使仲沄wsep次適皇兄寧武軍節度觀察留後孝騭wsep次在室',
 '孫三人wsep皆尚幼',
 '以崇寧四年正月十三日wsep葬於河南府洛陽縣淘牙村先塋之側',
 '銘曰wm\u3000\u3000苻氏之族wsep世有其後',

In [ ]:
grammer=r'''
         KIN1:
             {<nk><.+>+}
         '''
cp=nltk.RegexpParser(grammer)
string_list=replace_punct(df_qsw_raw_sample.iloc[8])
for string in string_list:
    sent = pseg.cut(string, HMM=True)
    kinship_string=cp.parse([(s,t) for s,t in sent])
    kin_list=[str(tree) for tree in kinship_string if str(tree).startswith('(KIN')]
    if len(kin_list)!=0:
        print(kin_list)

### Test tagging approach on sample.

### Do parallel computing on all QSW texts.

In [3]:
c = ipp.Client()
print(c.ids)
dview = c[:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


/tikal/ji/anaconda3/lib/python3.6/site-packages/ipyparallel/util.py:208: RuntimeWarning: IPython could not determine IPs for laxmi: [Errno -2] Name or service not known
  RuntimeWarning)
/tikal/ji/anaconda3/lib/python3.6/site-packages/ipyparallel/client/client.py:458: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@laxmi')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [4]:
df_qsw_raw=pd.read_excel('../data_raw/quan_song_wen_muzhi.xlsx', 
                         sheet_name='墓誌銘墓表壙誌行狀神道碑塔銘墓碑', 
                         encoding='utf8'
                        )
grammer=r'''
         KIN1:{<nk><.+><n|nr|nra>}
         KIN2:{<nk><n|nr|nra>}
         KIN3:{<nk|b><m>}
         KIN4:{<n|nr|nra><nk><n|nr|nra>}
         KIN5:{<a|m><v|vg><no.*><n|nr|nra.*>}
         KIN6:{<v|vg><no.*><n|nr|nra.*>}
         '''

In [5]:
dview.execute('import nltk')
dview.execute('import jieba as jb')
dview.execute('import jieba.posseg as pseg')
dview.execute("jb.load_userdict('../data_build/cn_traditional_jieba/nr_nra_dict_CBDB.txt')")
dview.execute("jb.load_userdict('../data_build/cn_traditional_jieba/ns_dict_CBDB.txt')")
dview.execute("jb.load_userdict('../data_build/cn_traditional_jieba/no_noc_dict_CBDB.txt')")
dview.execute("jb.load_userdict('../data_build/cn_traditional_jieba/td_tr_dict_CBDB.txt')")
dview.execute("jb.load_userdict('../data_build/cn_traditional_jieba/nk_dict_CBDB.txt')")
dview.execute("jb.load_userdict('../data_build/cn_traditional_jieba/nt_dict_CBDB.txt')")

<AsyncResult: execute>

In [6]:
dview['df_qsw_raw']=df_qsw_raw
dview['grammer']=grammer
dview['index_KinshipExtracted']=[]

In [7]:
@dview.parallel(block=True)
def kinship_extract(index_string):
    global grammer, index_KinshipExtracted
    index=index_string[0]
    string=index_string[1]
    try:
        cp=nltk.RegexpParser(grammer)
        sent = pseg.cut(string, HMM=True)
        kinship_string=cp.parse([(s,t) for s,t in sent])
        index_KinshipExtracted.append([index, ';'.join([str(tree) for tree in kinship_string if str(tree).startswith('(KIN')])])
    except Exception as error_msg:
        index_KinshipExtracted.append([index, 'EXCEPTION:'+str(error_msg)])

In [8]:
df_qsw_raw.reset_index(inplace=True)
t=kinship_extract.map(df_qsw_raw[['index', 'content']].values)

In [9]:
df_kinship_extract=pd.DataFrame(dview.gather('index_KinshipExtracted').get(), 
                                columns=['index', 'kinship_extract']).set_index('index')

In [11]:
df_qsw_merge=pd.concat([df_kinship_extract, 
                        df_qsw_raw], axis=1).drop(['index'], axis=1)
df_qsw_merge.to_excel('../data_clean/qsw_kinship_extracted.xlsx')